In [48]:
from __future__ import print_function, absolute_import
from WISE_tools import *
import FATS, pandas as pd, numpy as np
from multiprocessing import Pool

In [3]:
unique_names = parse_source_names('test_data/')

In [49]:
def get_features(name):
    """
    A function that is given a name, and then outputs all 68 features (plus the name)
    in a handy pandas dataframe.
    
    Parameter
    ---------
    Name : str
        WISE source name output by WISE_tools.parse_source_names
        
    Returns
    -------
    features : `~pandas.DataFrame`
        Dataframe of all 68 features, plus a column for the name.
    """
    
    df = get_lightcurve(name, 'test_data/')
    lc = np.array([df['w1mpro'].values,df['mjd'].values,df['w1sigmpro'].values,
                df['w2mpro'].values,df['w1mpro'].values,df['w2mpro'].values,
                df['mjd'].values,df['w1sigmpro'].values,df['w2sigmpro'].values])
    
    a = FATS.FeatureSpace(Data=['magnitude', 'time', 'error', 'magnitude2', 'error2'],
                           excludeList=[])
    a.calculateFeature(lc)
    result = a.result('dict')
    result['Name'] = name
    return pd.DataFrame(data=np.array(result.values()).reshape(1,len(result.values())), 
                        columns=result.keys())

In [56]:
%%time
p = Pool(8)
dfs = p.map(get_features,np.tile(unique_names,8))
out = pd.concat(dfs)
out.to_csv('test_features.csv')

CPU times: user 220 ms, sys: 129 ms, total: 349 ms
Wall time: 1min 7s
